# Document Generation
In this notebook we will begin by creating several documents (probably word docs) and writing them to a databricks volume for parsing. These documents will be generated samples of corrective actions that need to be distilled based on previous output from a different system.

## Setup Block
We're going to create a number of cells that handle the setup and maintenance of the data and files for the project. Although this can be done anywhere I find that putting the configs as a dedicated section at the top of my notebook easy to manage. What matters here is the system. Since serverless pipelines don't support `%run` commands (they're declarative) it's best to decouple functions in other ways.

In [0]:
%pip install python-docx mlflow  --upgrade --pre

dbutils.library.restartPython()

In [0]:
#Python utilities
import os, json, uuid, requests, datetime, re
from docx import Document
from pathlib import Path

#Spark utilities
from pyspark.sql import functions as F, types as T

#Databricks utilities
from dbruntime.databricks_repl_context import get_context

In [0]:
#Set the variables for the PAT in the Databricks Secrets store
secret_scope_name = "general"
secret_key_name = "genie_access"

#Inject the variables into the agent for use
os.environ["DB_MODEL_SERVING_HOST_URL"] = "https://" + get_context().workspaceUrl
assert os.environ["DB_MODEL_SERVING_HOST_URL"] is not None

#Inject the databricks personal access token for use
os.environ["DATABRICKS_GENIE_PAT"] = dbutils.secrets.get(
    scope=secret_scope_name, key=secret_key_name
)
assert os.environ["DATABRICKS_GENIE_PAT"] is not None, (
    "The DATABRICKS_GENIE_PAT was not properly set to the PAT secret"
)

In [0]:
#Keeping this separate makes it easy to find - it's the most likely to need to get updated on a frequent basis
catalog = "ademianczuk"
db = "suncor_ehs"

In [0]:
#Set the operating environment details
DATABRICKS_HOST = os.environ.get("DB_MODEL_SERVING_HOST_URL")
DATABRICKS_TOKEN = os.environ.get("DATABRICKS_GENIE_PAT")
FM_ENDPOINT = "llama-3-70b-instruct"  # your foundation model endpoint name

#Set the storage details
VOL_ROOT = f"/Volumes/{catalog}/{db}/data"
DOC_OUT_DIR = f"{VOL_ROOT}/docs"
Path(DOC_OUT_DIR).mkdir(parents=True, exist_ok=True)

#Set the table details
SCENARIO_TABLE = f"{catalog}.{db}.scenarios"
DOCS_TABLE = f"{catalog}.{db}.docs"

## Helper Functions
The two helper functions assist us in creating the contents for the document as well as writing them out to a storage volume. We may want to adjust the format of the word file to include tables as the real output will likely have. The point of this is to be able to reuse or extend this logic later. Since we may have a sample document, modelling off of that might be useful.

### Decoupling Helper Functionality
This might be worth porting over to a class file depending on how often it needs to be used.

In [0]:
#These operations are functionalized to make recall easier. We are leveraging globally assigned variables here. If we decouple this, the variables will need to be added to the signature and class constructor.

def call_chat(messages, temperature=0.2, max_tokens=1200):
    """
    Calls Databricks Foundation Model endpoint (chat-style).
    API schema per Foundation Model REST API docs.
    """
    url = f"{DATABRICKS_HOST}/api/2.0/serving-endpoints/{FM_ENDPOINT}/invocations"
    headers = {"Authorization": f"Bearer {DATABRICKS_TOKEN}", "Content-Type": "application/json"}
    payload = {
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    r = requests.post(url, headers=headers, data=json.dumps(payload), timeout=120)
    r.raise_for_status()
    resp = r.json()
    
    # Databricks FM APIs mirror OpenAI-like schema; adjust if your endpoint returns a different shape.
    # Try common fields first:
    content = None
    if isinstance(resp, dict):
        # 'choices' structure
        choices = resp.get("choices")
        if choices and len(choices) > 0:
            msg = choices[0].get("message") or {}
            content = msg.get("content")
    
    return content or str(resp)

def write_docx(title, actions_md, out_path):
    doc = Document()
    doc.add_heading(title, level=1)
    
    for line in actions_md.splitlines():
        if line.strip().startswith("- "):
            doc.add_paragraph(line.strip()[2:], style=None)
        else:
            doc.add_paragraph(line)
    
    doc.save(out_path)

In [0]:
# Option A: manually seed 10–15 "circumstances"
manual_scenarios = [
    {"scenario_id": str(uuid.uuid4()), "title": "CNC mill overheating at low spindle speeds",
     "context": {"machine":"CNC mill","symptoms":["process temp rising","reduced coolant flow"],"env":"ambient 32°C",
                 "likely_modes":["Heat Dissipation Failure (HDF)"]}},
    {"scenario_id": str(uuid.uuid4()), "title": "Assembly press showing increased vibration",
     "context": {"machine":"hydraulic press","symptoms":["vibration 2x baseline","oil temp high"],"env":"normal",
                 "likely_modes":["Component Wear","Hydraulic cavitation"]}},
    # ...add ~10-15
]

spark.createDataFrame(
    [(s["scenario_id"], s["title"], json.dumps(s["context"]), "manual") for s in manual_scenarios],
    schema="scenario_id string, title string, context_json string, source string"
).write.mode("overwrite").saveAsTable(SCENARIO_TABLE)

# Option B: (optional) Add synthetic from AI4I 2020 if you've loaded it to a table
# ai4i_df = spark.read.csv(f"{VOL_ROOT}/ai4i_2020.csv", header=True, inferSchema=True)
# # Create a few diverse conditions as scenarios:
# synth = (ai4i_df
#          .withColumn("scenario_id", F.expr("uuid()"))
#          .withColumn("title", F.concat(F.lit("AI4I condition: setting="), F.col("Type")))
#          .withColumn("context_json", F.to_json(F.struct(*ai4i_df.columns)))
#          .withColumn("source","ai4i_sample")
#          .limit(10))
# synth.write.mode("append").saveAsTable(SCENARIO_TABLE)

In [0]:
gen_system = """You are a maintenance reliability engineer.
Produce a detailed, **actionable** corrective action plan for the given machinery circumstance.
Write steps that a qualified technician can execute, with parts, tools, checks, and acceptance criteria.
Return:
- A 2–3 paragraph context summary.
- A prioritized list of corrective actions (10–20 items), each with:
  - Rationale
  - Skill level (Apprentice/Tech/Senior)
  - Est. downtime saved (hours)
  - Est. cost band ($, $$, $$$)
  - Safety notes
Finish with a short 'Verification & Re-start Procedure' checklist.
Use neutral, professional tone."""

scenarios_df = spark.table(SCENARIO_TABLE).limit(15).collect()
outputs = []
for row in scenarios_df:
    ctx = json.loads(row.context_json)
    user_prompt = f"""Circumstance Title: {row.title}
Context JSON:\n{json.dumps(ctx, indent=2)}"""

    content = call_chat([
        {"role":"system","content": gen_system},
        {"role":"user","content": user_prompt}
    ], temperature=0.2, max_tokens=1600)

    file_name = f"{row.title.lower().replace(' ','_')[:60]}_{row.scenario_id[:8]}.docx"
    file_path = f"{DOC_OUT_DIR}/{file_name}"
    write_docx(row.title, content, file_path)

    outputs.append((row.scenario_id, row.title, file_path, datetime.datetime.utcnow().isoformat(), content))

spark.createDataFrame(outputs, "scenario_id string, title string, docx_path string, created_utc string, raw_text string")\
     .write.mode("overwrite").saveAsTable(DOCS_TABLE)

print(f"Generated {len(outputs)} documents → {DOC_OUT_DIR}")

In [0]:
sum_system = """You are an expert maintenance prioritization analyst.
Given a corrective-action document, extract each action and score it using this rubric:
1) RiskReduction (0-5), 2) DowntimeAvoided (0-5), 3) CostEffectiveness (0-5), 4) TimeToImplement (0-5, invert score so faster=5), 5) Repeatability (0-5).
Compute ImpactScore = 0.35*RiskReduction + 0.25*DowntimeAvoided + 0.20*CostEffectiveness + 0.10*TimeToImplement + 0.10*Repeatability.
Return JSON with fields: actions: [{title, justification, scores:{...}, ImpactScore}], plus a brief summary."""

docs = spark.table(DOCS_TABLE).collect()
rank_rows = []
for d in docs:
    summary = call_chat([
        {"role":"system","content": sum_system},
        {"role":"user","content": d.raw_text[:120000]}  # keep under context window
    ], temperature=0.0, max_tokens=1200)

    # Optional: light validation
    m = re.search(r"\{.*\}", summary, flags=re.S)
    json_blob = m.group(0) if m else "{}"

    rank_rows.append((d.scenario_id, d.title, d.docx_path, json_blob))

schema = "scenario_id string, title string, docx_path string, ranking_json string"
spark.createDataFrame(rank_rows, schema).write.mode("overwrite").saveAsTable("main.corrective_actions.rankings")

print("Ranking complete. Inspect table main.corrective_actions.rankings for top actions per doc.")